## 2018,2019 데이터 합쳐진 데이터프레임으로 오프라인 장바구니 만들기

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale, minmax_scale, robust_scale
%matplotlib inline
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# 데이터 시각화 추세 패키지
import seaborn as sns

!pip3 install mlxtend
# 데이터 시각화 패키지
import matplotlib
import matplotlib.pyplot as plt
# 데이터 시각화 추세 패키지
import seaborn as sns
# 한글 패치
matplotlib.rc('font', family='Malgun Gothic')
matplotlib.rc('font', family='Malgun Gothic')
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
# 회귀분석
from statsmodels.formula.api import ols
# qqplot, 회귀분석의 상수항 추가
from statsmodels.api import qqplot, add_constant
# 선형 모델 formula(y~X1+X2+...)
import statsmodels.formula.api as smf
# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 선형회귀모형
from sklearn.linear_model import LinearRegression
# 변수 선택법(후진제거법)
from sklearn.feature_selection import RFE
# Scale 변환 -> 표준화 회귀계수 산출
from sklearn.preprocessing import StandardScaler
# 평가함수
# export_graphviz : 나무 구조 생성 및 저장
from sklearn.tree import export_graphviz
# graphviz : 나무 구조 시각화 (.dot 확장자 파일 불러오기 등)
import graphviz

# 데이터 분할 : train, test
from sklearn.model_selection import train_test_split
#예측/회귀 Decision tree
from sklearn.tree import DecisionTreeRegressor
# 예측 / 회귀 Random Forest
from sklearn.ensemble import RandomForestRegressor
#예측/회귀 GradientBoosting
from sklearn.ensemble import GradientBoostingRegressor
# 최적모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV
# os 환경
import os






from statsmodels.tools.eval_measures import rmse
from statsmodels.tools.eval_measures import mse
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true - y_pred))
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

In [2]:
member = pd.read_csv('C:/Users/ParkChihyun/Desktop/3주차 교육자료/빅데이터프로젝트/jeju_member.csv')
member.head()

,customer_id,birth_date,gender,age,agegroup,customer_type,customer_grade,gaip_date,taltoi_date,member_month,gaip_path,flag_member,flag_rcv_sms,flag_rcv_email,flag_rcv_post,zipcode,sigun,dong,ri,address
0,C102666,1976-11-08,남,44,40대,일반,3Gold,2014-10-07,NaN,63,매장,Y,Y,Y,Y,690-170,제주시,연동,NaN,제주 제주시 연동
1,C118113,1975-08-18,여,45,40대,일반,3Gold,2011-12-06,NaN,97,기타,Y,Y,Y,Y,690-122,제주시,아라2동,NaN,제주 제주시 아라2동
2,C118133,1981-03-20,남,39,30대,일반,4Silver,2011-12-07,2014-04-02,28,매장,N,Y,Y,N,699-900,서귀포시,성산읍,NaN,제주 서귀포시 성산읍
3,C118278,1979-06-22,여,41,40대,일반,3Gold,2011-12-11,NaN,97,인터넷,Y,Y,Y,Y,695-791,제주시,애월읍,광령2리,제주 제주시 애월읍 광령2리 제주관광대학
4,C118280,1959-08-28,남,61,60대,일반,3Gold,2011-12-11,2016-05-02,53,기타,N,N,N,N,699-905,서귀포시,성산읍,삼달리,제주 서귀포시 성산읍 삼달리


In [6]:
product = pd.read_csv('C:/Users/ParkChihyun/Desktop/빅데이터프로젝트/new_dataset/jeju_sales_off.csv',  encoding='cp949')
product.head()

,Unnamed: 0,customer_id,purchase_id,store_id,purchase_yearmonth,purchase_weekday,purchase_hour,purchase_date,purchase_time,supply_date,item_id,item_g_l,item_g_m,item_g_s,item_name,purchase_cnt,unit_cost,purchase_amt
0,0,C118113,C118113_20180110_87,제주매장,2018M01,수요일,14,2018-01-10,:0:14:47:04,2018-01-10,P08010117,간식,과자,스낵,꼬마찹쌀모나카(112g/8개),1,3000,3000
1,1,C118113,C118113_20180110_87,제주매장,2018M01,수요일,14,2018-01-10,:0:14:47:04,2018-01-10,P09040101,양념/가루,장,간장,제주전통어간장(500ml),1,7900,7900
2,2,C118113,C118113_20180110_87,제주매장,2018M01,수요일,14,2018-01-10,:0:14:47:04,2018-01-10,P09040100,양념/가루,장,간장,맛간장(900ml),1,13600,13600
3,3,C118113,C118113_20181008_22,제주매장,2018M10,월요일,11,2018-10-08,:0:11:59:24,2018-10-08,P09010305,양념/가루,가루,곡식가루,엿기름가루(무/500g),2,3200,6400
4,4,C118113,C118113_20181008_22,제주매장,2018M10,월요일,11,2018-10-08,:0:11:59:24,2018-10-08,P09010301,양념/가루,가루,곡식가루,메주가루(500g),2,11000,22000


In [7]:
date_number = set()
for i in product[['purchase_date', 'customer_id']].values:
    date_number.add(tuple(i))    # datenumber라는 세트만들고 그 안에 구매날짜와 id를 튜플형태로 넣는다.

In [8]:
# 빈 데이터프레임 생성
cart = pd.DataFrame(columns=['purchase_date', 'customer_id', '장바구니', 'purchase_cnt', 'purchase_amt', 'store_id']) 


# tqdm(프로그래스바), enumerate(세트에서 한개의 튜플을 차례로 꺼낸다.)
for num, data in tqdm(enumerate(date_number)):
    cart.loc[num, 'purchase_date'] = data[0]
    cart.loc[num, 'customer_id'] = data[1]
    # product data중에서 cart의 값들과 일치하는 값들만 가져와서 temp에 가져온다.
    temp = product.loc[(product['purchase_date'] == data[0]) & (product['customer_id'] == data[1])]
    # temp(같은 날짜, 같은 아이디)의 장바구니 대분류들을 모두 '장바구니_대'칼럼에 저장한다.
    cart.loc[num, '장바구니_대'] = '+'.join(temp['item_g_l'])
    cart.loc[num, '장바구니_중'] = '+'.join(temp['item_g_m'])
    cart.loc[num, '장바구니_소'] = '+'.join(temp['item_g_s'])
    cart.loc[num, '구매수량합계'] = temp['purchase_cnt'].sum()
    cart.loc[num, '구매금액합계'] = temp['purchase_amt'].sum()
    cart.loc[num, '구매매장'] = '+'.join(temp['store_id'])

In [9]:
cart

,purchase_date,customer_id,장바구니,purchase_cnt,purchase_amt,store_id,장바구니_대,장바구니_중,장바구니_소,구매수량합계,구매금액합계,구매매장
0,2018-10-20,C322512,NaN,NaN,NaN,NaN,반찬+간식+채소+축산물+축산물,밑반찬+과자+열매채소+유제품+유정란,김가공+스낵+고추+우유/산양유+유정란,7.0,17400.0,제주매장+제주매장+제주매장+제주매장+제주매장
1,2018-11-24,C567453,NaN,NaN,NaN,NaN,간식+축산물,식사대용+유제품,씨리얼+우유/산양유,3.0,10600.0,제주매장+제주매장
2,2018-01-16,C508629,NaN,NaN,NaN,NaN,축산물+간식,유제품+과자,우유/산양유+스낵,2.0,6200.0,제주매장+제주매장
3,2019-10-19,C427836,NaN,NaN,NaN,NaN,간식,밀가공,라면,1.0,5400.0,서귀포매장
4,2018-02-12,C302390,NaN,NaN,NaN,NaN,양념/가루+간식+간식+채소+양념/가루+채소+채소+반찬+반찬+채소+반찬+채소,기름/식초+빵+빵+양념채소+가루+버섯+잎채소+어묵/묵+밑반찬+잎채소+두부/유부+잎채소,기름+빵+빵+마늘+과채가루+느타리버섯+홍화나물+어묵+김가공+쑥+두부+콩나물,12.0,48790.0,제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제...
...,...,...,...,...,...,...,...,...,...,...,...,...
59489,2018-07-05,C357557,NaN,NaN,NaN,NaN,채소+채소+양념/가루+축산물+축산물,열매채소+열매채소+장+육가공+유제품,고추+오이+된장+돼지가공+요구르트,6.0,27500.0,제주매장+제주매장+제주매장+제주매장+제주매장
59490,2019-07-11,C508477,NaN,NaN,NaN,NaN,양념/가루+축산물+반찬+축산물+양념/가루,장+한우+김치+유제품+소스/기타양념,된장+안심대안한우+배추+요구르트+소스,5.0,46330.0,제주매장+제주매장+제주매장+제주매장+제주매장
59491,2018-12-04,C548461,NaN,NaN,NaN,NaN,축산물+축산물+간식+간식+채소+반찬+간식+축산물+반찬+채소+채소+채소+채소+간식+간...,육가공+육가공+과자+과자+잎채소+밑반찬+밀가공+돼지+어묵/묵+잎채소+잎채소+잎채소+...,돼지가공+돼지가공+스낵+스낵+쌈채소모음+김가공+라면+보리살림돼지+어묵+깻잎+얼갈이+...,22.0,58150.0,제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제...
59492,2019-09-22,C510586,NaN,NaN,NaN,NaN,간식+채소,과자+버섯,스낵+기타,2.0,80100.0,제주매장+제주매장


In [10]:
cart.to_csv('장바구니offline.csv', index=False)

## 카트에 분기 칼럼 추가

In [11]:
df= pd.DataFrame(cart['purchase_date'].str[5:7]) # 월 떼어오기
df=df.apply(pd.to_numeric)
df=df.astype({'purchase_date': int})

df['quarter']='4분기'
df.rename(columns = {'purchase_date' : 'month'}, inplace = True)

for i in range(len(df)):
    if df['month'][i]==1:
        df['quarter'][i]='1분기'
    if df['month'][i]==2:
        df['quarter'][i]='1분기'
    if df['month'][i]==3:
        df['quarter'][i]='1분기'
    if df['month'][i]==4:
        df['quarter'][i]='2분기'
    if df['month'][i]==5:
        df['quarter'][i]='2분기'
    if df['month'][i]==6:
        df['quarter'][i]='2분기'
    if df['month'][i]==7:
        df['quarter'][i]='3분기'
    if df['month'][i]==8:
        df['quarter'][i]='3분기'
    if df['month'][i]==9:
        df['quarter'][i]='3분기'
        
cart['month']=df['month']
cart['quarter']=df['quarter']

cart

<ipython-input-11-7d5919addb4e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quarter'][i]='1분기'
<ipython-input-11-7d5919addb4e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quarter'][i]='1분기'
<ipython-input-11-7d5919addb4e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quarter'][i]='3분기'
<ipython-input-11-7d5919addb4e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,purchase_date,customer_id,장바구니,purchase_cnt,purchase_amt,store_id,장바구니_대,장바구니_중,장바구니_소,구매수량합계,구매금액합계,구매매장,month,quarter
0,2018-10-20,C322512,NaN,NaN,NaN,NaN,반찬+간식+채소+축산물+축산물,밑반찬+과자+열매채소+유제품+유정란,김가공+스낵+고추+우유/산양유+유정란,7.0,17400.0,제주매장+제주매장+제주매장+제주매장+제주매장,10,4분기
1,2018-11-24,C567453,NaN,NaN,NaN,NaN,간식+축산물,식사대용+유제품,씨리얼+우유/산양유,3.0,10600.0,제주매장+제주매장,11,4분기
2,2018-01-16,C508629,NaN,NaN,NaN,NaN,축산물+간식,유제품+과자,우유/산양유+스낵,2.0,6200.0,제주매장+제주매장,1,1분기
3,2019-10-19,C427836,NaN,NaN,NaN,NaN,간식,밀가공,라면,1.0,5400.0,서귀포매장,10,4분기
4,2018-02-12,C302390,NaN,NaN,NaN,NaN,양념/가루+간식+간식+채소+양념/가루+채소+채소+반찬+반찬+채소+반찬+채소,기름/식초+빵+빵+양념채소+가루+버섯+잎채소+어묵/묵+밑반찬+잎채소+두부/유부+잎채소,기름+빵+빵+마늘+과채가루+느타리버섯+홍화나물+어묵+김가공+쑥+두부+콩나물,12.0,48790.0,제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제...,2,1분기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59489,2018-07-05,C357557,NaN,NaN,NaN,NaN,채소+채소+양념/가루+축산물+축산물,열매채소+열매채소+장+육가공+유제품,고추+오이+된장+돼지가공+요구르트,6.0,27500.0,제주매장+제주매장+제주매장+제주매장+제주매장,7,3분기
59490,2019-07-11,C508477,NaN,NaN,NaN,NaN,양념/가루+축산물+반찬+축산물+양념/가루,장+한우+김치+유제품+소스/기타양념,된장+안심대안한우+배추+요구르트+소스,5.0,46330.0,제주매장+제주매장+제주매장+제주매장+제주매장,7,3분기
59491,2018-12-04,C548461,NaN,NaN,NaN,NaN,축산물+축산물+간식+간식+채소+반찬+간식+축산물+반찬+채소+채소+채소+채소+간식+간...,육가공+육가공+과자+과자+잎채소+밑반찬+밀가공+돼지+어묵/묵+잎채소+잎채소+잎채소+...,돼지가공+돼지가공+스낵+스낵+쌈채소모음+김가공+라면+보리살림돼지+어묵+깻잎+얼갈이+...,22.0,58150.0,제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제...,12,4분기
59492,2019-09-22,C510586,NaN,NaN,NaN,NaN,간식+채소,과자+버섯,스낵+기타,2.0,80100.0,제주매장+제주매장,9,3분기


In [12]:
cart

,purchase_date,customer_id,장바구니,purchase_cnt,purchase_amt,store_id,장바구니_대,장바구니_중,장바구니_소,구매수량합계,구매금액합계,구매매장,month,quarter
0,2018-10-20,C322512,NaN,NaN,NaN,NaN,반찬+간식+채소+축산물+축산물,밑반찬+과자+열매채소+유제품+유정란,김가공+스낵+고추+우유/산양유+유정란,7.0,17400.0,제주매장+제주매장+제주매장+제주매장+제주매장,10,4분기
1,2018-11-24,C567453,NaN,NaN,NaN,NaN,간식+축산물,식사대용+유제품,씨리얼+우유/산양유,3.0,10600.0,제주매장+제주매장,11,4분기
2,2018-01-16,C508629,NaN,NaN,NaN,NaN,축산물+간식,유제품+과자,우유/산양유+스낵,2.0,6200.0,제주매장+제주매장,1,1분기
3,2019-10-19,C427836,NaN,NaN,NaN,NaN,간식,밀가공,라면,1.0,5400.0,서귀포매장,10,4분기
4,2018-02-12,C302390,NaN,NaN,NaN,NaN,양념/가루+간식+간식+채소+양념/가루+채소+채소+반찬+반찬+채소+반찬+채소,기름/식초+빵+빵+양념채소+가루+버섯+잎채소+어묵/묵+밑반찬+잎채소+두부/유부+잎채소,기름+빵+빵+마늘+과채가루+느타리버섯+홍화나물+어묵+김가공+쑥+두부+콩나물,12.0,48790.0,제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제...,2,1분기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59489,2018-07-05,C357557,NaN,NaN,NaN,NaN,채소+채소+양념/가루+축산물+축산물,열매채소+열매채소+장+육가공+유제품,고추+오이+된장+돼지가공+요구르트,6.0,27500.0,제주매장+제주매장+제주매장+제주매장+제주매장,7,3분기
59490,2019-07-11,C508477,NaN,NaN,NaN,NaN,양념/가루+축산물+반찬+축산물+양념/가루,장+한우+김치+유제품+소스/기타양념,된장+안심대안한우+배추+요구르트+소스,5.0,46330.0,제주매장+제주매장+제주매장+제주매장+제주매장,7,3분기
59491,2018-12-04,C548461,NaN,NaN,NaN,NaN,축산물+축산물+간식+간식+채소+반찬+간식+축산물+반찬+채소+채소+채소+채소+간식+간...,육가공+육가공+과자+과자+잎채소+밑반찬+밀가공+돼지+어묵/묵+잎채소+잎채소+잎채소+...,돼지가공+돼지가공+스낵+스낵+쌈채소모음+김가공+라면+보리살림돼지+어묵+깻잎+얼갈이+...,22.0,58150.0,제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제주매장+제...,12,4분기
59492,2019-09-22,C510586,NaN,NaN,NaN,NaN,간식+채소,과자+버섯,스낵+기타,2.0,80100.0,제주매장+제주매장,9,3분기


## 1분기의 장바구니 분석

In [17]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['quarter'] == '1분기', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=0.1, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
14,0.299,(잎채소)
2,0.280,(두부/유부)
1,0.241,(과자)
4,0.185,(밀가공)
12,0.169,(유제품)
13,0.168,(육가공)
10,0.163,(양념채소)
5,0.149,(밑반찬)
7,0.145,(빵)
8,0.128,(뿌리채소)


In [18]:
df_asso_rule = association_rules(df_freq, metric='confidence', min_threshold=0.3)
df_asso_rule.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(두부/유부),(잎채소),0.280086,0.299264,0.117595,0.419851,1.402946,0.033775,1.207856
1,(잎채소),(두부/유부),0.299264,0.280086,0.117595,0.392946,1.402946,0.033775,1.185914


## 2분기의 장바구니 분석

In [24]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['quarter'] == '2분기', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=0.05, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
20,0.266,(잎채소)
4,0.254,(두부/유부)
2,0.231,(과자)
18,0.196,(유제품)
7,0.191,(밀가공)
17,0.178,(유정란)
16,0.171,(열매채소)
19,0.158,(육가공)
10,0.149,(빵)
1,0.136,(과일채소)


In [25]:
df_asso_rule = association_rules(df_freq, metric='confidence', min_threshold=0.3)
df_asso_rule.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9,(뿌리채소),(잎채소),0.110357,0.266255,0.052803,0.478475,1.797055,0.023420,1.406922
3,(열매채소),(잎채소),0.170965,0.266255,0.080969,0.473601,1.778749,0.035449,1.393894
8,(양념채소),(잎채소),0.121080,0.266255,0.053957,0.445628,1.673689,0.021719,1.323560
10,(과일채소),(잎채소),0.136487,0.266255,0.052599,0.385380,1.447411,0.016259,1.193820
0,(두부/유부),(잎채소),0.253970,0.266255,0.087349,0.343934,1.291746,0.019728,1.118401
6,(열매채소),(두부/유부),0.170965,0.253970,0.057282,0.335054,1.319262,0.013862,1.121939
1,(잎채소),(두부/유부),0.266255,0.253970,0.087349,0.328065,1.291746,0.019728,1.110271
5,(유정란),(잎채소),0.178227,0.266255,0.057418,0.322163,1.209979,0.009964,1.082480
7,(유정란),(두부/유부),0.178227,0.253970,0.056740,0.318355,1.253512,0.011475,1.094455
4,(밀가공),(두부/유부),0.190647,0.253970,0.059454,0.311855,1.227918,0.011036,1.084116


## 3분기의 장바구니 분석

In [30]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['quarter'] == '3분기', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=0.05, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
4,0.256,(두부/유부)
16,0.242,(열매채소)
18,0.210,(유제품)
2,0.205,(과자)
7,0.204,(밀가공)
20,0.198,(잎채소)
17,0.170,(유정란)
19,0.163,(육가공)
10,0.141,(빵)
8,0.138,(밑반찬)


In [31]:
df_asso_rule = association_rules(df_freq, metric='confidence', min_threshold=0.3)
df_asso_rule.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(잎채소),(열매채소),0.197882,0.242057,0.088714,0.448318,1.852114,0.040815,1.373876
8,(양념채소),(열매채소),0.124781,0.242057,0.051074,0.409311,1.690968,0.020870,1.283151
1,(열매채소),(잎채소),0.242057,0.197882,0.088714,0.366500,1.852114,0.040815,1.266169
4,(잎채소),(두부/유부),0.197882,0.256097,0.067655,0.341896,1.335026,0.016978,1.130373
3,(열매채소),(두부/유부),0.242057,0.256097,0.081755,0.337750,1.318837,0.019765,1.123297
6,(유정란),(열매채소),0.170408,0.242057,0.056339,0.330611,1.365836,0.015090,1.132290
2,(두부/유부),(열매채소),0.256097,0.242057,0.081755,0.319234,1.318837,0.019765,1.113368
5,(밀가공),(두부/유부),0.203873,0.256097,0.064206,0.314930,1.229731,0.011995,1.085879
7,(유정란),(두부/유부),0.170408,0.256097,0.051498,0.302202,1.180029,0.007857,1.066072


## 4분기의 장바구니 분석

In [35]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['quarter'] == '4분기', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=0.05, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
3,0.256,(두부/유부)
20,0.249,(잎채소)
1,0.199,(과자)
19,0.168,(육가공)
17,0.155,(유정란)
7,0.154,(밀가공)
18,0.150,(유제품)
10,0.148,(빵)
14,0.138,(양념채소)
8,0.137,(밑반찬)


In [36]:
df_asso_rule = association_rules(df_freq, metric='confidence', min_threshold=0.3)
df_asso_rule.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(뿌리채소),(잎채소),0.126911,0.248681,0.053714,0.423241,1.701944,0.022154,1.302657
2,(양념채소),(잎채소),0.138209,0.248681,0.055676,0.402839,1.619904,0.021306,1.258152
1,(잎채소),(두부/유부),0.248681,0.256393,0.084630,0.340316,1.327320,0.020870,1.127216
0,(두부/유부),(잎채소),0.256393,0.248681,0.084630,0.330079,1.327320,0.020870,1.121504
